In [91]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

In [4]:
model = YOLO('best.pt')

In [67]:
video_path = "D:AI/Top View Car Parked Place Disabled Stock Footage Video (100% Royalty-free) 1106163301.mp4"

generator = sv.get_video_frames_generator(video_path)
iterator = iter(generator)
frame = next(iterator)

In [69]:
cv2.imwrite('first_frame.jpg',frame)

True

In [74]:
polygons = [
    np.array([[74, 282], [891, 275], [887, 496], [74, 495]]),
    np.array([[74, 150], [893, 140], [891, 17], [70, 33]])
]
zones = [sv.PolygonZone(polygon=polygon)
        for polygon in polygons]

In [75]:
zone_annotators = [sv.PolygonZoneAnnotator(zone=zone,color=sv.Color.BLUE,thickness=1,text_thickness=1,
                                         text_scale=2)for _,zone in enumerate(zones)]
box_annotators = [sv.BoxAnnotator(thickness=2) for _ in zones]


In [89]:
def process_frame(frame: np.array,_) -> np.array:
    results = model(frame,imgsz=1280)[0]
    detections = sv.Detections.from_ultralytics(results)
    
    for zone , zone_annotator , box_annotator in zip (zones,zone_annotators,box_annotators):
        mask = zone.trigger(detections=detections)
        detection_filtered = detections[mask]
        
        frame = box_annotator.annotate(scene=frame,detections=detection_filtered)
        frame = zone_annotator.annotate(scene=frame)
    return frame

In [95]:
cap = cv2.VideoCapture(video_path)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = process_frame(frame, None)

    cv2.imshow('Real-Time Video', processed_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

from IPython.display import clear_output
clear_output(wait=True)